In [1]:
import csv
import random
from utils import *
from collections import defaultdict, Counter
import os
import time
import tqdm
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np

## generate SBMs and Erdos-Renyi graph

In [2]:
class SBMGraphStream():
    '''
    The class of Graph Stream from the stochastic block model
    ----- Parameters -----
    # n_vertex: the number of vertices in the graph
    # p_intra: the probability for + edge (u,v) for the same cluster
    # p_inter: the probability for + edge (u,v) for different clusters
    # k_cluster: number of clusters in the clustering
    ----- Methods ----
    # read_next_edge(): read the next edge and move the index +1
    ----- Representation ----
    The graph is representation with an indexed array of vertices and a dictionary with (u_i, u_j): labels
    '''
    
    def __init__(self, n_vertex, p_intra=0.8, p_inter=0.2, k_cluster=7):
        '''
        :param n_vertex: the the number of vertices in the graph
        '''
        self.n_vertex = n_vertex
        self.p_intra = p_intra
        self.p_inter = p_inter
        self.k_cluster = k_cluster
        
        # initialize the vertex set and the cluster labels
        self.vertex_set = np.array([self.n_vertex])
        num_v_per_cluster = n_vertex//self.k_cluster
        n_residual = n_vertex % num_v_per_cluster
        cluster_labels_list = []
        for i_cluster in range(k_cluster):
            cluster_labels_list.append(i_cluster*np.ones([num_v_per_cluster]))
        if n_residual!=0:
            cluster_labels_list.append((k_cluster-1)*np.ones([n_residual]))
        # collect them as a 1-d array
        self.cluster_labels = np.reshape(np.hstack(cluster_labels_list).astype(int), [-1])
        # initialize the edges -- using +1 and -1 to represent the edge labels
        # also compute the cost
        self.cc_cost = 0
        self.edge_dict = {}
        for u_i in tqdm.tqdm(range(self.n_vertex)):
            for u_j in np.arange(u_i+1, self.n_vertex):
                if self.cluster_labels[u_i] == self.cluster_labels[u_j]:
                    if np.random.rand() <= p_intra:
                        self.edge_dict[(u_i,u_j)] = 1
                    else:
                        self.edge_dict[(u_i,u_j)] = -1
                        self.cc_cost = self.cc_cost + 1
                else:
                    if np.random.rand() <= p_inter:
                        self.edge_dict[(u_i,u_j)] = 1
                        self.cc_cost = self.cc_cost + 1
                    else:
                        self.edge_dict[(u_i,u_j)] = -1
        # randomize the order of edge arrival
        self.edge_names = list(self.edge_dict.keys())
        random.shuffle(self.edge_names)
        self.num_edges = len(self.edge_names)
        # maintain a pointer of the number of edges
        self.current_stream_ind = 0
        
    def read_next_edge(self):
        
        this_edge_name = self.edge_names[self.current_stream_ind]
        this_edge_label = self.edge_dict[this_edge_name]
        self.current_stream_ind = self.current_stream_ind + 1
        if self.current_stream_ind>=self.num_edges-1:
            return None, None
        
        return this_edge_name, this_edge_label
    
    def write_edges(self, write_path=None):
        if not write_path:
            raise ValueError('the writing path has to be specified!')
        file_name = 'SBM_n='+str(self.n_vertex)+'_p='+str(self.p_intra)+'_k=' + str(self.k_cluster) +'.csv'
        with open(os.path.join(write_path, file_name), 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for edge in self.edge_dict:
                if self.edge_dict[edge]>0:
                    writer.writerow([f'{edge[0]} {edge[1]}'])
    
    def reset_index(self):
        '''
        reset the pointer
        '''
        self.current_stream_ind = 0

In [3]:
sbm_graph_stream = SBMGraphStream(n_vertex=200,p_intra=0.95, p_inter=0.05, k_cluster=4)

100%|██████████| 200/200 [00:00<00:00, 5671.20it/s]


In [4]:
sbm_graph_stream.write_edges(write_path='../data/')

## The functions that implements our algorithm

In [5]:
def sparsity_test(target_vertex, current_graph, eps=0.2):
    current_num_vertex = len(current_graph)+1
    num_sample = max((int)(np.log(current_num_vertex)/eps), 20)
    num_diff_test_sample = max((int)(np.log(current_num_vertex)/eps), 20)
    if current_graph[target_vertex].degree == 0:
        return True
    num_neighbor_diff = 0
    for comp_vertex in current_graph[target_vertex].getRandom(i=num_diff_test_sample):
        if current_graph[comp_vertex].degree>(1+eps)*current_graph[target_vertex].degree:
            num_neighbor_diff = num_neighbor_diff + 1
            continue
        total_diff = 0
        for test_vertex in current_graph[target_vertex].getRandom(i=num_sample):
            if test_vertex not in current_graph[comp_vertex]:
                total_diff = total_diff + 1
        if (total_diff>=eps*num_diff_test_sample):
            num_neighbor_diff = num_neighbor_diff + 1
    if num_neighbor_diff >=eps*num_diff_test_sample:
        return True
    else:
        return False

In [6]:
def test_sparse_vertex(current_graph, global_graph, eps=0.2):
    # sample log n neighbors for each vertex
    sparse_vertex_list = []
    for this_vertex in current_graph:
        if sparsity_test(this_vertex, global_graph, eps=eps):
            sparse_vertex_list.append(this_vertex)
        
    # return the list of sparse vertices
    return sparse_vertex_list

In [41]:
def sparse_dense_decop(current_graph, global_graph, eps=0.2):
    current_num_vertex = len(global_graph)+1
    # the returned clusters
    SDD_clustering = {}
    # check sparse vertices
    tic = time.time()
    current_sparse_vertice = test_sparse_vertex(current_graph, global_graph, eps=eps)
    for sparse_vertex in current_sparse_vertice:
        SDD_clustering[sparse_vertex]=sparse_vertex
    # print('The time for sparsity testing is ', time.time()-tic)
    # sample from the dense vertices
    dense_subgraph = {vertex: current_graph[vertex] 
                      for vertex in current_graph if vertex not in current_sparse_vertice}
    anchor_vertex_dict = {}
    # tic = time.time()
    for this_vertex in dense_subgraph:
        # rejection sampling
        dense_sample_prob = max((np.log(current_num_vertex))/(eps*dense_subgraph[this_vertex].degree), 0.02)
        if dense_sample_prob>=np.random.uniform(low=0.0, high=1.0):
            anchor_vertex_dict[this_vertex] = dense_subgraph[this_vertex]
    # print('The time for indexing the subgraph ', time.time()-tic)
    # recursively form almost-cliques
    num_sample = max((int)(5*np.log(current_num_vertex)), 20)
    AC_dict = {}
    # maintain a list of covered vertices
    # tic = time.time()
    covered_AC_vertex = []
    for this_anchor_vertex in anchor_vertex_dict.keys():
        if this_anchor_vertex in covered_AC_vertex:
            continue
        AC_dict[this_anchor_vertex] = {}
        AC_dict[this_anchor_vertex]['AC'] = []
        AC_dict[this_anchor_vertex]['size'] = 1
        AC_dict[this_anchor_vertex]['counter'] = 0
        SDD_clustering[this_anchor_vertex] = this_anchor_vertex  # assign to the cluster represented by self
        covered_AC_vertex.append(this_anchor_vertex)
        anchor_neighbor_samples = list(anchor_vertex_dict[this_anchor_vertex].getRandom(i=num_sample))
        for candidate_vertex in dense_subgraph[this_anchor_vertex]:
            if (candidate_vertex in current_sparse_vertice) or (candidate_vertex in covered_AC_vertex):
                continue
            # test whether their symmetric difference is large enough
            # the intersections of the neighbor
            total_diff = 0
            for anchor_neighbor in anchor_neighbor_samples:
                if anchor_neighbor not in list(dense_subgraph[candidate_vertex]):
                    total_diff = total_diff + 1
            if (total_diff<=1.5*eps*num_sample):
                AC_dict[this_anchor_vertex]['AC'].append(candidate_vertex)
                AC_dict[this_anchor_vertex]['size'] += 1
                SDD_clustering[candidate_vertex] = this_anchor_vertex # assign the candidate vertex to the anchor
                covered_AC_vertex.append(candidate_vertex)
            # this line is for debugging purpose -- remove later
            else:
                pass
    # add codes to add vertices to the almost-cliques
    # print('The time for forming almost-cliques is ', time.time()-tic)
    # merge undecided vertices to the almost-cliques
    num_diff_test_sample = max((int)(2*np.log(current_num_vertex)), 20)
    # pull the undecided vertices
    undecided_AC_vertices = []
    for dense_vertex in dense_subgraph.keys():
        if dense_vertex not in SDD_clustering.keys():
            undecided_AC_vertices.append(dense_vertex)
    for und_vertex in undecided_AC_vertices:
        und_v_sampled_neighbors = list(dense_subgraph[und_vertex].getRandom(i=num_diff_test_sample))
        neighbor_AC_names = []
        for merge_test_vertex in und_v_sampled_neighbors:
            if merge_test_vertex not in SDD_clustering:
                continue
            neighbor_AC_names.append(SDD_clustering[merge_test_vertex])
        if not neighbor_AC_names:
            current_sparse_vertice.append(und_vertex)
            SDD_clustering[und_vertex] = und_vertex
            continue
        AC_counts = Counter(neighbor_AC_names)
        most_common_AC, most_common_AC_freq = AC_counts.most_common(1)[0]
#         except:
#             print(neighbor_AC_names)
#             raise ValueError('Something wrong just happened!')
        if (most_common_AC_freq>=(1-2*eps)*num_diff_test_sample) and (most_common_AC in AC_dict):
            SDD_clustering[und_vertex] = most_common_AC
            AC_dict[most_common_AC]['AC'].append(und_vertex)
            AC_dict[most_common_AC]['size'] += 1
        else:
            # this line should not happen in theory, but we need to handle singleton cases in practice
            current_sparse_vertice.append(und_vertex)
            SDD_clustering[und_vertex] = und_vertex
    
    
    return current_sparse_vertice, AC_dict, SDD_clustering, anchor_vertex_dict

In [42]:
def singleton_cluster_alg(current_graph):
    return {vertex: vertex for vertex in current_graph}

## The implementation of dynamic algorithms

In [43]:
def extract_induced_subgraph(current_graph, target_vertex):
    num_vertex = len(current_graph)+1
    neighbor_of_target = list(current_graph[target_vertex]) + [target_vertex]
    num_sample = min(max((int)(2*np.log(num_vertex)), 20), len(neighbor_of_target))
    induced_subgraph = {}
    valid_neighbor_vertices = []
    for neighbor_vertex in neighbor_of_target:
        this_neighbor_list = list(set(current_graph[neighbor_vertex].a).intersection(set(neighbor_of_target)))
        if len(this_neighbor_list)>=0.3*current_graph[neighbor_vertex].degree:
            valid_neighbor_vertices.append(neighbor_vertex)
    for neighbor_vertex in valid_neighbor_vertices:
        # let's pretend that intersection happens in O(1) time
        this_new_neighbor_list = list(set(current_graph[neighbor_vertex].a).intersection(set(valid_neighbor_vertices)))
        induced_subgraph[neighbor_vertex] = OptList(a=this_new_neighbor_list,
                                                    d=this_new_neighbor_list, 
                                                    degree=current_graph[neighbor_vertex].degree)
        # estimation of the cost
#         sample_vertex_in_subgraph = random.sample(neighbor_of_target, num_sample)
#         count_degree = 0
#         for sampled_vertex in sample_vertex_in_subgraph:
#             if sampled_vertex in current_graph[neighbor_vertex]:
#                 count_degree = count_degree + 1
#         est_degree = count_degree*len(neighbor_of_target)/num_sample
#         induced_subgraph[neighbor_vertex] = OptList(a=this_neighbor_list, d=this_neighbor_list, degree=est_degree)
    
    return induced_subgraph

In [44]:
def dynamic_sparse_dense_decomp(subgraph, global_graph, global_SDD, 
                                global_sparse_vertices, global_AC_dict, target_vertex, eps=0.2):
    '''
    params: 
    subgraph: the subgraph that we want to run SDD on. Note that this is *not* the exact N(u) since we
                filter out all the vertices that are not dense locally
    global_graph: the global graph represented by the dictionary of OptList()
    global_SDD: the sparse-dense decomposition we maintain for the global graph (clustering representation)
    global_sparse_vertices: the list of sparse vertices
    target_vertex: we need this information to run the sparsity tests for all vertices in N(u)
    '''
    num_vertex = len(global_graph)
    # ************** Formation of Almost-cliques *******************
    subgraph_sparse_vertex_list,subgraph_AC_dict, subgraph_SDD_clustering, anchor_vertex_dict = sparse_dense_decop(
        subgraph, global_graph, eps=eps)
    # update the almost-cliques and the list of sparse vertices
    # this test should not be necessary in theory
    # just to avoid weird bugs
    new_almost_cliques = {}
    for this_anchor_vertex in subgraph_AC_dict:
        this_AC_anchor_dict = subgraph_AC_dict[this_anchor_vertex]
        new_almost_cliques[this_anchor_vertex] = {}
        new_almost_cliques[this_anchor_vertex]['AC'] = []
        new_almost_cliques[this_anchor_vertex]['size'] = 1
        new_almost_cliques[this_anchor_vertex]['counter'] = 0
        this_AC_anchor_neighbors = this_AC_anchor_dict['AC']
        this_AC_size = this_AC_anchor_dict['size']
        num_invalid_vertex = 0
        for neighbor_vertex in this_AC_anchor_neighbors:
            if (global_graph[neighbor_vertex].degree 
                >= (1+2*eps)*this_AC_size) or (neighbor_vertex not in global_sparse_vertices):
                num_invalid_vertex += 1
                this_AC_anchor_dict['AC'].remove(neighbor_vertex)
                this_AC_anchor_dict['size'] = this_AC_anchor_dict['size'] - 1
            elif neighbor_vertex in global_sparse_vertices:
                global_sparse_vertices.remove(neighbor_vertex)
            else:
                print('Something strange happened; vertex', neighbor_vertex,'should be a globally sparse vertex.')
                pass
        # dismantle the AC if too many members have been lost
        if num_invalid_vertex>=2*eps*this_AC_size:
            global_sparse_vertices.append(this_AC_anchor_dict)
            for member_vertex in this_AC_anchor_neighbors:
                global_sparse_vertices.append(member_vertex)
        # add new vertices
        num_sample_from_AC = max((int)(3*np.log(num_vertex)), 20)
        num_sample_tests_AC = max((int)(3*np.log(num_vertex)/eps), 20)
        if num_sample_from_AC>len(this_AC_anchor_neighbors):
            sample_AC_vertex_list = this_AC_anchor_neighbors[:]
        else:
            sample_AC_vertex_list = random.sample(this_AC_anchor_neighbors, num_sample_from_AC)
        if num_sample_tests_AC>len(this_AC_anchor_neighbors):
            sampled_T_set = this_AC_anchor_neighbors[:]
        else:
            sampled_T_set = random.sample(this_AC_anchor_neighbors, num_sample_tests_AC)
        # find the neighbors of these vertices
        for sampled_AC_vertex in sample_AC_vertex_list:
            neighbors_of_the_sampled_vertex = global_graph[sampled_AC_vertex]
            for candidate_vertex in neighbors_of_the_sampled_vertex:
                total_agreement = 0
                for sample_T_vertex in sampled_T_set:
                    if sample_T_vertex in global_graph[candidate_vertex]:
                        total_agreement = total_agreement +1
                if (total_agreement>=(1-2*eps)*num_sample_tests_AC):
                    if (global_graph[candidate_vertex].degree>=(1-2*eps)*this_AC_size):
                        if (global_graph[candidate_vertex].degree<=(1+2*eps)*this_AC_size):
                            this_AC_anchor_dict['AC'].append(candidate_vertex)
                            this_AC_anchor_dict['size'] += 1
        new_almost_cliques[this_anchor_vertex]['AC'] = this_AC_anchor_dict['AC']
        new_almost_cliques[this_anchor_vertex]['size'] = this_AC_anchor_dict['size']
    # merge the almost-cliques with the existing SDD
    # the sparse vertex list has already been updated
    for new_dense_anchor in new_almost_cliques:
        global_AC_dict[new_dense_anchor] = new_almost_cliques[new_dense_anchor]
        for AC_member in new_almost_cliques[new_dense_anchor]['AC']:
            global_SDD[AC_member] = new_dense_anchor
    # ******************** Removing Vertices from Almost-cliques ***********************
    neighbors_of_the_target_vertex = global_graph[target_vertex]
    for possible_candidate_sparse in neighbors_of_the_target_vertex:
        if sparsity_test(possible_candidate_sparse, global_graph, eps=eps):
            if possible_candidate_sparse in global_sparse_vertices:
                continue
            # add to the list of sparse vertices
            global_sparse_vertices.append(possible_candidate_sparse)
            global_SDD[possible_candidate_sparse] = possible_candidate_sparse
            # remove from almost-cliques
            AC_anchor = global_SDD[possible_candidate_sparse]
            if AC_anchor not in global_AC_dict:
                global_sparse_vertices.append(AC_anchor)
                global_SDD[AC_anchor] = AC_anchor
                continue
            if global_AC_dict[AC_anchor]['size'] == 1:
                global_sparse_vertices.append(AC_anchor)
                global_SDD[AC_anchor] = AC_anchor
                del global_AC_dict[AC_anchor]
                continue
            if possible_candidate_sparse == AC_anchor:
                first_other_anchor = global_AC_dict[AC_anchor]['AC'][0]
                global_AC_dict[first_other_anchor] = {}
                global_AC_dict[first_other_anchor]['AC'] = []
                global_AC_dict[first_other_anchor]['size'] = global_AC_dict[AC_anchor]['size'] - 1
                global_AC_dict[first_other_anchor]['counter'] = global_AC_dict[AC_anchor]['counter']
                for member in global_AC_dict[AC_anchor]['AC']:
                    if member!=first_other_anchor:
                        global_AC_dict[first_other_anchor]['AC'].append(member)
                global_sparse_vertices.append(possible_candidate_sparse)
                global_SDD[possible_candidate_sparse] = possible_candidate_sparse
                continue
            global_AC_dict[AC_anchor]['AC'].remove(possible_candidate_sparse)
#             except:
#                 print('++++++++++++++++++++++++++++')
#                 print(possible_candidate_sparse)
#                 print(global_SDD[possible_candidate_sparse])
#                 print(global_AC_dict[AC_anchor]['size'])
#                 print(global_AC_dict[AC_anchor]['AC'])
#                 raise ValueError('Something wrong just happened!')
            global_AC_dict[AC_anchor]['counter'] = global_AC_dict[AC_anchor]['counter'] + 1
            # **************** Dismantle almost-cliques if they loss too many vertices
            if global_AC_dict[AC_anchor]['counter']>=1.2*eps*global_AC_dict[AC_anchor]['size']:
                global_sparse_vertices.append(AC_anchor)
                global_SDD[AC_anchor] = AC_anchor
                for past_dense_vertex in global_AC_dict[AC_anchor]['AC']:
                    global_sparse_vertices.append(past_dense_vertex)
                    global_SDD[past_dense_vertex] = past_dense_vertex
                del global_AC_dict[AC_anchor]
    
    return global_sparse_vertices, global_AC_dict, global_SDD, anchor_vertex_dict

## Read the edges and maintain clustering

In [45]:
# graph_file_name="../data/SBM_n=200_p=0.95_k=4.csv"
graph_file_name= "../data/email-Eu-core.csv"

In [46]:
adjacency_list, edge_list = create_graph_from_csv(graph_file_name)

In [47]:
'''
TODO

See if functions are doable

'''

'\nTODO\n\nSee if functions are doable\n\n'

In [48]:
no_edges = len(edge_list)  # No. of edges

prob_del = 0.2      # Probability to delete edge
eps_param = 0.8

current_graph = {}
current_edge_list = []

available_edge_list = np.random.permutation(edge_list).tolist()

stream_length = (int)(0.5*no_edges)

track_update_num = {}
track_update_benckmark = {}

max_ite = 10000

SDD_amortized_update_time = 0
pivot_amortized_update_time = 0

# initialize the SDD clustering with singletons
SDD_clustering = singleton_cluster_alg(adjacency_list)
current_sparse_vertex_list = list(adjacency_list.keys())
almost_cliques = {}

for i in range(stream_length):
    # Insertion
    if available_edge_list: #
        current_edge_list.append(available_edge_list[i])
        u = available_edge_list[i][0]
        v = available_edge_list[i][1]
        if u not in current_graph.keys():
            current_graph[u] = OptList()
        current_graph[u].insert(v)
        if v not in current_graph.keys():
            current_graph[v] = OptList()
        current_graph[v].insert(u)
        available_edge_list.pop(0)
        
        # keep track of the benchmark for the updates
        if u not in track_update_benckmark:
            track_update_benckmark[u] = current_graph[u].degree
        if v not in track_update_benckmark:
            track_update_benckmark[v] = current_graph[v].degree
        # update the tracking of the updates on u and v
        if u not in track_update_num:
            track_update_num[u] = 1
        else:
            track_update_num[u] = track_update_num[u] + 1
            
        if v not in track_update_num:
            track_update_num[v] = 1
        else:
            track_update_num[v] = track_update_num[v] + 1
         
        '''
        Code for SDD and PIVOT goes here
        '''
        # SDD clustering
        # Maintain the original clustering -- if the newly added vertice are not in the SDD
        # then we assign singletons
        if u not in SDD_clustering:
            SDD_clustering[u] = u
        if v not in SDD_clustering:
            SDD_clustering[v] = v
        # The update step
        if (track_update_num[u]>max(2, eps_param*track_update_benckmark[u])):
            # tests
            current_subgraph = extract_induced_subgraph(current_graph, u)
            if len(current_subgraph) == 1 and (not list(current_subgraph[list(current_subgraph.keys())[0]])):
                continue
            start_SDD = time.time()
            current_sparse_vertex_list, almost_cliques, SDD_clustering, anchor_vertex_dict = dynamic_sparse_dense_decomp(current_subgraph,
                                                                                                                         current_graph, 
                                                                                                                         SDD_clustering,
                                                                                                                         current_sparse_vertex_list,
                                                                                                                         almost_cliques, 
                                                                                                                         u,
                                                                                                                         eps=eps_param)
            SDD_amortized_update_time = (time.time() - start_SDD)/track_update_num[u]
            track_update_num[u] = 0
            track_update_benckmark[u] = current_graph[u].degree
        start_pivot = time.time()
        pivot_clustering = classical_pivot(current_graph)
        end_pivot = time.time()
        pivot_amortized_update_time = end_pivot - start_pivot
        singleton_clustering = singleton_cluster_alg(adjacency_list)
        # clear the number of updates
        # track_update_num[u] = 0
        if i>1000:
            SDD_cost = correlation_clustering_value(current_graph, SDD_clustering)
#             all_vertex_list = [v for v in current_graph]
#             AC_vertex_list = []
#             for anchor_ver in almost_cliques.keys():
#                 AC_vertex_list.append(anchor_ver)
#                 for ac_ver in almost_cliques[anchor_ver]:
#                     AC_vertex_list.append(ac_ver)
#             AC_vertex_list = list(set(AC_vertex_list))
#             recovered_vertex = np.concatenate((AC_vertex_list, current_sparse_vertex_list))
#             print('===============================')
#             print(np.setdiff1d(all_vertex_list,recovered_vertex))
#             print(len(current_sparse_vertex_list))
#             print(len(anchor_vertex_dict.keys()))
#             print('******************************')
#             print('The number of almost-cliques is ', len(almost_cliques))
            pivot_cost = correlation_clustering_value(current_graph, pivot_clustering)
            singleton_cost = correlation_clustering_value(current_graph, singleton_clustering)
            print('SDD clustering cost is', SDD_cost, 'and the running time is', SDD_amortized_update_time)
            print('Pivot clustering cost is', pivot_cost, 'and the running time is', pivot_amortized_update_time)
            print('Singleton clustering cost is', singleton_cost)
#         if "SBM" in graph_file_name:
#             print('The correct optimal clustering cost should be', sbm_graph_stream.cc_cost)
        if i>max_ite:
            all_vertex_list = [v for v in current_graph]
            AC_vertex_list = []
            for anchor_ver in almost_cliques.keys():
                AC_vertex_list.append(anchor_ver)
                for ac_ver in almost_cliques[anchor_ver]:
                    AC_vertex_list.append(ac_ver)
            AC_vertex_list = list(set(AC_vertex_list))
            recovered_vertex = np.concatenate((AC_vertex_list, current_sparse_vertex_list))
            print('===============================')
            print(np.setdiff1d(all_vertex_list,recovered_vertex))
            print(len(current_sparse_vertex_list))
            print(len(anchor_vertex_dict.keys()))
            print('******************************')
            print('The number of almost-cliques is ', len(almost_cliques))
            break
        
#     else:
#         # We have run out of edges to insert
#         edge_to_delete = np.random.choice(current_edge_list)
        
#         u = edge_to_delete[0]
#         v = edge_to_delete[1]
#         current_graph[u].remove(v)
#         current_graph[v].remove(u)
        
#         available_edge_list.extend(edge_to_delete)
#         current_edge_list.remove(edge_to_delete)
    
        
    
    
#     if np.random.binomial(1,prob_del):
#         # Deletion
#         print(current_edge_list)
#         edge_to_delete = np.random.choice(current_edge_list)
        
#         u = edge_to_delete[0]
#         v = edge_to_delete[1]
#         current_graph[u].remove(v)
#         current_graph[v].remove(u)
        
#         available_edge_list.extend(edge_to_delete)
#         current_edge_list.remove(edge_to_delete)print('++++++++++++++++++++++++++++')

SDD clustering cost is 1114.5 and the running time is 8.900960286458333e-05
Pivot clustering cost is 1144.0 and the running time is 0.0007588863372802734
Singleton clustering cost is 965.0
SDD clustering cost is 1115.5 and the running time is 8.900960286458333e-05
Pivot clustering cost is 1184.0 and the running time is 0.0007622241973876953
Singleton clustering cost is 966.0
SDD clustering cost is 1116.5 and the running time is 5.7220458984375e-05
Pivot clustering cost is 1271.5 and the running time is 0.0008780956268310547
Singleton clustering cost is 967.0
SDD clustering cost is 1117.5 and the running time is 6.230672200520833e-05
Pivot clustering cost is 1378.5 and the running time is 0.0008609294891357422
Singleton clustering cost is 968.0
SDD clustering cost is 1118.5 and the running time is 6.230672200520833e-05
Pivot clustering cost is 1241.0 and the running time is 0.0008418560028076172
Singleton clustering cost is 969.0
SDD clustering cost is 1119.5 and the running time is 6.2

SDD clustering cost is 1222.0 and the running time is 0.00017905235290527344
Pivot clustering cost is 1211.5 and the running time is 0.0011792182922363281
Singleton clustering cost is 1034.0
SDD clustering cost is 1223.0 and the running time is 0.00017905235290527344
Pivot clustering cost is 1265.0 and the running time is 0.0008268356323242188
Singleton clustering cost is 1035.0
SDD clustering cost is 1224.0 and the running time is 0.00017905235290527344
Pivot clustering cost is 1383.5 and the running time is 0.0009210109710693359
Singleton clustering cost is 1036.0
SDD clustering cost is 1225.0 and the running time is 0.00017905235290527344
Pivot clustering cost is 1221.5 and the running time is 0.0008182525634765625
Singleton clustering cost is 1037.0
SDD clustering cost is 1230.5 and the running time is 5.805492401123047e-05
Pivot clustering cost is 1221.0 and the running time is 0.0008339881896972656
Singleton clustering cost is 1038.0
SDD clustering cost is 1231.5 and the running 

SDD clustering cost is 1317.5 and the running time is 4.172325134277344e-05
Pivot clustering cost is 1434.0 and the running time is 0.0007989406585693359
Singleton clustering cost is 1107.0
SDD clustering cost is 1318.5 and the running time is 4.172325134277344e-05
Pivot clustering cost is 1380.0 and the running time is 0.0007958412170410156
Singleton clustering cost is 1108.0
SDD clustering cost is 1319.5 and the running time is 4.172325134277344e-05
Pivot clustering cost is 1353.0 and the running time is 0.0007898807525634766
Singleton clustering cost is 1109.0
SDD clustering cost is 1319.5 and the running time is 0.00010429109845842634
Pivot clustering cost is 1401.5 and the running time is 0.0007860660552978516
Singleton clustering cost is 1109.0
SDD clustering cost is 1320.5 and the running time is 0.00010429109845842634
Pivot clustering cost is 1697.0 and the running time is 0.0007808208465576172
Singleton clustering cost is 1110.0
SDD clustering cost is 1314.5 and the running ti

SDD clustering cost is 1377.5 and the running time is 0.00019044876098632811
Pivot clustering cost is 1422.5 and the running time is 0.0007989406585693359
Singleton clustering cost is 1179.0
SDD clustering cost is 1380.5 and the running time is 0.00017853577931722006
Pivot clustering cost is 1518.5 and the running time is 0.0008029937744140625
Singleton clustering cost is 1180.0
SDD clustering cost is 1382.5 and the running time is 5.5670738220214844e-05
Pivot clustering cost is 1480.0 and the running time is 0.0008490085601806641
Singleton clustering cost is 1182.0
SDD clustering cost is 1383.5 and the running time is 0.00014702479044596353
Pivot clustering cost is 1552.0 and the running time is 0.0008189678192138672
Singleton clustering cost is 1183.0
SDD clustering cost is 1387.0 and the running time is 6.874402364095052e-05
Pivot clustering cost is 1454.5 and the running time is 0.0008058547973632812
Singleton clustering cost is 1184.0
SDD clustering cost is 1388.0 and the running 

SDD clustering cost is 1434.0 and the running time is 9.686606270926339e-05
Pivot clustering cost is 1501.5 and the running time is 0.0008220672607421875
Singleton clustering cost is 1236.0
SDD clustering cost is 1430.0 and the running time is 9.918212890625e-05
Pivot clustering cost is 1559.5 and the running time is 0.0008730888366699219
Singleton clustering cost is 1237.0
SDD clustering cost is 1431.0 and the running time is 0.00010001659393310547
Pivot clustering cost is 1574.5 and the running time is 0.0008168220520019531
Singleton clustering cost is 1238.0
SDD clustering cost is 1436.0 and the running time is 7.802248001098633e-05
Pivot clustering cost is 1459.0 and the running time is 0.0009882450103759766
Singleton clustering cost is 1239.0
SDD clustering cost is 1437.0 and the running time is 7.802248001098633e-05
Pivot clustering cost is 1588.0 and the running time is 0.0008409023284912109
Singleton clustering cost is 1240.0
SDD clustering cost is 1438.0 and the running time i

...

SDD clustering cost is 14190.0 and the running time is 0.00021986734299432663
Pivot clustering cost is 9263.5 and the running time is 0.001438140869140625
Singleton clustering cost is 7561.0
SDD clustering cost is 14191.0 and the running time is 0.00021986734299432663
Pivot clustering cost is 9138.0 and the running time is 0.0014309883117675781
Singleton clustering cost is 7562.0
SDD clustering cost is 14192.0 and the running time is 0.00021986734299432663
Pivot clustering cost is 9040.5 and the running time is 0.0014328956604003906
Singleton clustering cost is 7563.0
SDD clustering cost is 14191.5 and the running time is 5.173683166503906e-05
Pivot clustering cost is 8927.0 and the running time is 0.0014629364013671875
Singleton clustering cost is 7563.0
SDD clustering cost is 14192.5 and the running time is 5.173683166503906e-05
Pivot clustering cost is 8541.5 and the running time is 0.0014367103576660156
Singleton clustering cost is 7564.0
SDD clustering cost is 14192.5 and the runn

SDD clustering cost is 14224.5 and the running time is 0.00022064149379730225
Pivot clustering cost is 10314.5 and the running time is 0.0014491081237792969
Singleton clustering cost is 7596.0
SDD clustering cost is 14224.5 and the running time is 0.00022064149379730225
Pivot clustering cost is 9643.5 and the running time is 0.0014331340789794922
Singleton clustering cost is 7596.0
SDD clustering cost is 14225.5 and the running time is 0.00022064149379730225
Pivot clustering cost is 9755.0 and the running time is 0.0015339851379394531
Singleton clustering cost is 7597.0
SDD clustering cost is 14225.5 and the running time is 0.00022064149379730225
Pivot clustering cost is 8876.5 and the running time is 0.0018389225006103516
Singleton clustering cost is 7597.0
SDD clustering cost is 14226.5 and the running time is 0.00022064149379730225
Pivot clustering cost is 12435.0 and the running time is 0.001416921615600586
Singleton clustering cost is 7598.0
SDD clustering cost is 14013.5 and the 

SDD clustering cost is 13593.5 and the running time is 0.00013549327850341796
Pivot clustering cost is 9288.0 and the running time is 0.0014638900756835938
Singleton clustering cost is 7627.0
SDD clustering cost is 13594.5 and the running time is 0.00013549327850341796
Pivot clustering cost is 10269.5 and the running time is 0.0014560222625732422
Singleton clustering cost is 7628.0
SDD clustering cost is 13595.5 and the running time is 0.00013549327850341796
Pivot clustering cost is 9823.0 and the running time is 0.0014638900756835938
Singleton clustering cost is 7629.0
SDD clustering cost is 13596.5 and the running time is 0.00013549327850341796
Pivot clustering cost is 9011.5 and the running time is 0.001531839370727539
Singleton clustering cost is 7630.0
SDD clustering cost is 13596.5 and the running time is 0.00013549327850341796
Pivot clustering cost is 9389.0 and the running time is 0.0014798641204833984
Singleton clustering cost is 7630.0
SDD clustering cost is 13597.5 and the r

SDD clustering cost is 13804.5 and the running time is 0.00038393338521321613
Pivot clustering cost is 8991.5 and the running time is 0.001489877700805664
Singleton clustering cost is 7659.0
SDD clustering cost is 13804.5 and the running time is 0.00038393338521321613
Pivot clustering cost is 9360.5 and the running time is 0.0014536380767822266
Singleton clustering cost is 7659.0
SDD clustering cost is 13805.5 and the running time is 0.00038393338521321613
Pivot clustering cost is 10332.0 and the running time is 0.0014369487762451172
Singleton clustering cost is 7660.0
SDD clustering cost is 13805.5 and the running time is 0.00038393338521321613
Pivot clustering cost is 9149.5 and the running time is 0.0014410018920898438
Singleton clustering cost is 7660.0
SDD clustering cost is 13805.5 and the running time is 0.00038393338521321613
Pivot clustering cost is 9673.5 and the running time is 0.0014998912811279297
Singleton clustering cost is 7660.0
SDD clustering cost is 13806.5 and the r

SDD clustering cost is 13834.5 and the running time is 0.00017563501993815103
Pivot clustering cost is 9250.0 and the running time is 0.0014929771423339844
Singleton clustering cost is 7691.0
SDD clustering cost is 13835.5 and the running time is 0.00017563501993815103
Pivot clustering cost is 10918.5 and the running time is 0.0014450550079345703
Singleton clustering cost is 7692.0
SDD clustering cost is 13836.5 and the running time is 0.00017563501993815103
Pivot clustering cost is 11034.0 and the running time is 0.0014338493347167969
Singleton clustering cost is 7693.0
SDD clustering cost is 13836.5 and the running time is 0.00017563501993815103
Pivot clustering cost is 10091.5 and the running time is 0.00144195556640625
Singleton clustering cost is 7693.0
SDD clustering cost is 13836.5 and the running time is 0.00017563501993815103
Pivot clustering cost is 10738.0 and the running time is 0.0014522075653076172
Singleton clustering cost is 7693.0
SDD clustering cost is 13837.5 and the

For of clustering is dict[vertex-name]: cluster-name